# Plotting the autocorrelation functions for the stars from Mathur et al. ([2014](https://www.mendeley.com/catalogue/investigating-magnetic-activity-f-stars-kepler-mission/)) and comparing observable peaks to the period values from the paper

Importing non-standard packages and modules:

[kplr](https://github.com/dfm/kplr), [autocorr](https://github.com/dfm/emcee/blob/master/src/emcee/autocorr.py) module from [emcee](https://github.com/dfm/emcee), and [plot_setup](https://github.com/dfm/celerite/blob/master/celerite/plot_setup.py) module from [celerite](https://github.com/dfm/celerite) developed by [Daniel Foreman-Mackey](https://github.com/dfm) and other contributors;<br>
[matplotlib](https://github.com/matplotlib/matplotlib) developed by [Michael Droettboom](https://github.com/mdboom),  [Thomas Caswell](https://github.com/tacaswell) and other contributors;<br>
[autograd](https://github.com/HIPS/autograd) developed by [Dougal Maclaurin](https://dougalmaclaurin.com), [David Duvenaud](https://www.cs.toronto.edu/~duvenaud/), [Matt Johnson](http://people.csail.mit.edu/mattjj/), [Jamie Townsend](https://github.com/j-towns) and other contributors.

In [4]:
import kplr # Tools for working with Kepler data
from emcee import autocorr as ac # Autocorrelation Functions
from celerite import plot_setup # Scalable 1D Gaussian Processes

from matplotlib import rcParams as rcP, pyplot as plt # Plots
from autograd import numpy as np # Derivatives of numpy code

Setting plots parameters:

In [ ]:
# A magic command to get inline plots within a Python Notebook
%matplotlib inline

# A magic command to enable 2x plots
%config IPython.matplotlib.backend = "retina"

# Setting DPIs of figures
rcP["savefig.dpi"] = 300
rcP["figure.dpi"]  = 300

# Setting other plots settings
plot_setup.setup()

Defining a function to load data from the [Kepler Input Catalog](https://archive.stsci.edu/kepler/kic.html):

In [5]:
def get_data(kic_id):
    """
    A function to get short cadence light curves data from a star
    with specified ID from the Kepler Input Catalog 
    
    Args:
        kic_id: The id of a star in the Kepler Input Catalog.
         
    Returns:
        array: Time values of the time series;
        array: Pre-search Data Conditioning Simple Aperture Photometry (PDCSAP) flux values.
        
    """

    # Attaching the kplr API
    client = kplr.API()

    # Getting the star data
    star = client.star(kic_id)

    # Time values
    t = []
    
    # Pre-search Data Conditioning Simple Aperture Photometry (PDCSAP) flux values
    f = []
    
    # Getting short cadence light curves
    for lc in star.get_light_curves(short_cadence=False):
        
        # Reading data from a light curve
        data = lc.read()
        
        # Getting time values from the data
        t0 = data["TIME"]
        
        # Getting PDCSAP flux values from the data
        f0 = data["PDCSAP_FLUX"]
        
        # Creating a mask for only qualitative data points
        mask = (data["SAP_QUALITY"] == 0) & np.isfinite(t0) & np.isfinite(f0)
    
        # Appending the qualitative time data to the existing list
        t.append(t0[mask])
        
        # Computing the median of the flux data
        mu = np.median(f0[mask])
        
        # Scaling the qualitative flux data and appending these to the existing list
        f.append( ( f0[mask] / mu - 1.0 ) * 1e3 )
        
    # Concatenating the arrays within the lists
    
    t = np.concatenate(t)
    f = np.concatenate(f)
    
    # Getting contiguous arrays
    
    t = np.ascontiguousarray(t, dtype = float)
    f = np.ascontiguousarray(f, dtype = float)
    
    # Returning arrays
    
    return t, f

Defining a function to calculate and plot the correlogram from the [Kepler](https://archive.stsci.edu/kepler/kic.html) data:

In [ ]:
def get_acf(t, f, kic_id, check_value):
    """
    A function to calculate and plot the correlogram of
    short cadence light curves data and to add the line of
    the period value taken from Mathur et al. (2014)
    
    Args:
        t: An array of time values;
        f: An array of flux values;
        kic_id: The id of a star in the Kepler Input Catalog;
        check_value: The period value taken for a star referred in Mathur et al. (2014)
    
    """
    
    # Calculating the median of the first order differences of the time array 
    delta_t = np.median(np.diff(t))
    
    # Interpolating values within the time array with delta_t spacing
    f_interp = np.interp(np.arange(t.min(), t.max(), delta_t), t, f)
    
    # Calculating unnormalized autocorrelation function
    corr = ac.function_1d(f_interp) * np.var(f_interp)

    # Defining a lag array
    tau = np.arange(len(corr)) * delta_t
    
    # Plotting data
    plt.plot(tau, corr)

    # Adding lines for specified checking period value 
    
    plt.axvline(check_value, color="k",ls="dashed", alpha=0.5)
    plt.axvline(2 * check_value, color="k",ls="dashed", alpha=0.5)
    plt.axvline(3 * check_value, color="k",ls="dashed", alpha=0.5)

    # Limitting the plot
    plt.xlim(0, 20)
    
    # Setting labels
    plt.ylabel(r"$\left< k(\tau) \right>$")
    plt.xlabel(r"$\tau$")

    # Saving the figure
    plt.savefig("figures/autocorrs/" + kic_id + ".png", bbox_inches="tight")

[KIC 1430163](https://archive.stsci.edu/kepler/data_search/search.php?target=1430163&action=Search&outputformat=HTML_Table): 3.88 $\pm$ 0.58 days
